# **Chapter 5: Support Vector Machines (SVM)**
- Klasifikasi Linear SVM (hard margin dan soft margin)
- Kernel trick dan klasifikasi SVM nonlinier
- Regresi SVM
- Online SVM dan hinge loss


# **Support Vector Machines**

SVM merupakan model yang sangat efektif untuk klasifikasi, regresi, dan deteksi outlier. Model ini berusaha menemukan margin selebar mungkin ("jalan") yang memisahkan kelas dengan memaksimalkan jarak antara decision boundary dan data latih terdekat yang disebut **support vectors**.



# **Klasifikasi Linear SVM**
- **Hard margin**: Pemisahan ketat tanpa kesalahan, hanya berfungsi bila data benar-benar linier terpisah dan sensitif terhadap outlier.
- **Soft margin**: Mengizinkan pelanggaran margin terbatas, dikendalikan oleh hyperparameter \( C \), menyeimbangkan antara ukuran margin dan kesalahan klasifikasi untuk generalisasi lebih baik.

### SVM Objective (Soft Margin)

$$ \min_{w,b,\zeta} \; \frac{1}{2} ||w||^2 + C \sum_{i=1}^{m} \zeta_i \quad \text{dengan syarat:} \quad y_i(w^T x_i + b) \geq 1 - \zeta_i,\; \zeta_i \geq 0 $$

### Fungsi prediksi SVM untuk instance baru \( x \):

$$ f(x) = \text{sign}(w^T x + b) $$

Training mencari \( w, b \) untuk memaksimalkan margin sambil meminimalkan pelanggaran.



In [4]:
# Contoh klasifikasi Linear SVM dengan dataset iris
from sklearn.datasets import load_iris
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

iris = load_iris()
X = iris.data[:, (2, 3)]  # panjang dan lebar kelopak
y = (iris.target == 2).astype(np.float64)  # klasifikasi iris virginica

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

svm_clf.fit(X, y)
print(svm_clf.predict([[5.5, 1.7]]))  # prediksi iris virginica

[1.]


# **Pentingnya Feature Scaling**

SVM sangat sensitif terhadap skala fitur. Selalu lakukan standardisasi fitur (mean = 0, varian = 1) untuk margin dan performa yang lebih baik.

# **SVM Nonlinear dan Kernel Trick**

Sebagian besar dataset tidak dapat dipisahkan secara linear. Dua pendekatan umum:
- Menambahkan fitur polinomial secara manual (mahal untuk derajat tinggi)
- Gunakan **kernel trick** untuk menghitung dot product dalam ruang berdimensi tinggi tanpa pemetaan eksplisit.

### Kernel umum:
- Linear: $$ K(a, b) = a^T b $$
- Polynomial: $$ K(a, b) = (\gamma a^T b + r)^d $$
- Gaussian RBF: $$ K(a, b) = \exp(-\gamma ||a - b||^2) $$
- Sigmoid: $$ K(a, b) = \tanh(\gamma a^T b + r) $$

In [5]:
# Contoh SVM nonlinier dengan kernel polinomial
from sklearn.datasets import make_moons
from sklearn.svm import SVC

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X, y)
print("Jumlah support vectors:", len(poly_kernel_svm_clf.named_steps["svm_clf"].support_))

Jumlah support vectors: 19


# **Gaussian RBF Kernel**

Kernel Gaussian (RBF) memetakan data ke ruang berdimensi tak hingga, memungkinkan SVM mempelajari batas keputusan kompleks hanya dengan sedikit hyperparameter:
- $$\gamma$$ seberapa besar pengaruh setiap titik
- \( C \): keseimbangan antara lebar margin dan pelanggaran

In [6]:
# Contoh SVM dengan kernel RBF
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=2, C=1))
])

rbf_kernel_svm_clf.fit(X, y)
print("Jumlah support vectors:", len(rbf_kernel_svm_clf.named_steps["svm_clf"].support_))

Jumlah support vectors: 36


# **Dual Problem dan Kernel Trick**

Pelatihan SVM dapat dinyatakan sebagai masalah optimasi kuadratik ganda, fokus pada support vector dan dot product antar data.

Dengan mengganti dot product dengan fungsi kernel, **kernel trick** memungkinkan pembelajaran dalam ruang fitur berdimensi tinggi tanpa transformasi eksplisit.

Efeknya: komputasi lebih efisien dan klasifikasi nonlinier bisa dilakukan dengan baik.

# **SVM untuk Regresi (SVR)**

SVM juga dapat digunakan untuk regresi dengan menyesuaikan batas kesalahan sebesar \( \varepsilon \) di sekitar fungsi prediksi. Nilai di luar batas ini dikenakan penalti.

Versi linier dan kernelisasi tersedia mirip dengan klasifikasi.


# Contoh regresi menggunakan SVR
from sklearn.svm import LinearSVR

X_reg = 2 * np.random.rand(100, 1)
y_reg = 4 + 3 * X_reg.ravel() + np.random.randn(100)

svr_reg = LinearSVR(epsilon=0.5, C=1.0, max_iter=10000)
svr_reg.fit(X_reg, y_reg)
print(f"Koefisien SVR: {svr_reg.coef_}")

# **Online SVM dan Hinge Loss**
SVM dapat dilatih secara bertahap (online) menggunakan stochastic gradient descent dan fungsi kerugian hinge loss:

$$ J(w, b) = \frac{1}{2} ||w||^2 + C \sum_i \max(0, 1 - y_i(w^T x_i + b)) $$

Fungsi loss ini menghukum prediksi yang salah atau terlalu dekat dengan margin.

Pelatihan online cocok untuk dataset besar atau bersifat streaming.

Untuk dataset sangat besar dan nonlinear, model seperti **neural network** sering lebih disarankan.
